In [125]:
# Import libraries

import os
import time
import warnings
import numpy as np
from datetime import date

from qiskit import IBMQ
from qiskit import pulse
from qiskit import assemble
from qiskit import schedule
from qiskit import QuantumCircuit
from qiskit.tools.jupyter import *
from qiskit.pulse import pulse_lib
from qiskit.compiler import transpile
from qiskit.tools.monitor import job_monitor

# Disble ignore warnings
warnings.filterwarnings('ignore')

In [126]:
# Load account and get backend info
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')

backend_config = backend.configuration()
assert backend_config.open_pulse, "Backend doesn't support OpenPulse"

# Get the discrete sampling time
dt = backend_config.dt
print(f'Sampling time: {dt*1e9} ns')

# Qubit to study
qubit = 0

# Find out which group of qubits need to be acquired with this qubit
meas_map_idx = None
for i, measure_group in enumerate(backend_config.meas_map):
    if qubit in measure_group:
        meas_map_idx = i
        break
assert meas_map_idx is not None, f"Couldn't find qubit {qubit} in the meas_map!"

ibmqfactory.load_account:WARNING:2020-05-23 13:09:22,797: Credentials are already in use. The existing account in the session will be replaced.


Sampling time: 0.2222222222222222 ns


In [127]:
# Global variables

# Unit conversion factors
GHz = 1.0e9 # Gigahertz
MHz = 1.0e6 # Megahertz
us = 1.0e-6 # Microseconds
ns = 1.0e-9 # Nanoseconds

# Number of samples
num_bins =10
num_samples_per_bin = 10
num_samples = 2*num_bins*num_samples_per_bin

# Number of shots
num_shots = 1024

# Max schedules
max_sched = 75

# Create directory
today = date.today()
today = str(today)
today = "%s-%s-%s"%(today.split('-')[0],today.split('-')[1],today.split('-')[2])

date_dir = '../data/' + today
if not os.path.exists(date_dir):
    os.makedirs(date_dir)

In [128]:
# Construct the schedules

train_o_p = []
train_cnt = [0]*num_bins
train_scd = []

valid_o_p = []
valid_cnt = [0]*num_bins
valid_scd = []

total_cnt = 0

while total_cnt != num_samples:
    
    # Develop the schedule
    qc = QuantumCircuit(qubit+1, qubit+1)
    a1 = np.random.uniform(-np.pi, np.pi)
    a2 = np.random.uniform(-np.pi, np.pi)
    a3 = np.random.uniform(-np.pi, np.pi)
    ins = qc.u3(a1, a2, a3, qubit)
    gate = ins.__getitem__(qubit)
    prob_0 = np.absolute(gate.to_matrix()[1][1])**2
    qc.measure(qubit, qubit)
    trans_qc = transpile(qc, backend)
    sched = schedule(trans_qc, backend)
    
    # Determine how to bin the schedule
    if train_cnt[int(prob_0*num_bins)] < num_samples_per_bin:
        train_o_p.append(prob_0)
        train_scd.append(sched)
        train_cnt[int(prob_0*num_bins)] += 1
        total_cnt += 1
    elif valid_cnt[int(prob_0*num_bins)] < num_samples_per_bin:
        valid_o_p.append(prob_0)
        valid_scd.append(sched)
        valid_cnt[int(prob_0*num_bins)] += 1
        total_cnt += 1

np.save(date_dir + '/train_o_p.npy', train_o_p)
np.save(date_dir + '/valid_o_p.npy', valid_o_p)

In [129]:
train_d_1 = []
train_d_2 = []
valid_d_1 = []
valid_d_2 = []

# Run the training level 1 program
ind = 0 
jobs = []
while True:
    mn = ind
    mx = min(ind + max_sched, len(train_scd))
    
    print(f'Now scheduling {mn} to {mx}.')
    
    schedul_program = assemble(train_scd[mn:mx],
                               backend=backend,
                               meas_level=1,
                               meas_return='single',
                               shots=num_shots)
    
    num_jobs_in_q = backend.status().pending_jobs
    print(f'Pending Jobs: {num_jobs_in_q}')
    
    job = backend.run(schedul_program)
    job_monitor(job)
    jobs.append(job)
    
    ind += max_sched
    
    if ind >= len(train_scd):
        break
        
# Get the training level 1 output
id1 = 0
id2 = 0
for itr in range(len(train_scd)):
    
    train_d_1.append(jobs[id1].result().get_memory(id2)[:, qubit] * 1e-14)
                
    id2 += 1
    if id2 == max_sched:
        id2 = 0
        id1 += 1

np.save(date_dir + '/train_d_1.npy', train_d_1)

# Run the training level 2 program
ind = 0 
jobs = []
while True:
    mn = ind
    mx = min(ind + max_sched, len(train_scd))
    
    print(f'Now scheduling {mn} to {mx}.')
    
    schedul_program = assemble(train_scd[mn:mx],
                               backend=backend,
                               meas_level=2,
                               meas_return='single',
                               shots=num_shots)
    
    num_jobs_in_q = backend.status().pending_jobs
    print(f'Pending Jobs: {num_jobs_in_q}')
    
    job = backend.run(schedul_program)
    job_monitor(job)
    jobs.append(job)
    
    ind += max_sched
    
    if ind >= len(train_scd):
        break

# Get the training level 2 output
id1 = 0
id2 = 0
for itr in range(len(train_scd)):
    
    train_d_2.append(jobs[id1].result().get_counts(id2)['0'])
                
    id2 += 1
    if id2 == max_sched:
        id2 = 0
        id1 += 1

np.save(date_dir + '/train_d_2.npy', train_d_2)

# Run the validation level 1 program
ind = 0 
jobs = []
while True:
    mn = ind
    mx = min(ind + max_sched, len(valid_scd))
    
    print(f'Now scheduling {mn} to {mx}.')
    
    schedul_program = assemble(valid_scd[mn:mx],
                               backend=backend,
                               meas_level=1,
                               meas_return='single',
                               shots=num_shots)
    
    num_jobs_in_q = backend.status().pending_jobs
    print(f'Pending Jobs: {num_jobs_in_q}')
    
    job = backend.run(schedul_program)
    job_monitor(job)
    jobs.append(job)
    
    ind += max_sched
    
    if ind >= len(valid_scd):
        break
        
# Get the validation level 1 output
id1 = 0
id2 = 0
for itr in range(len(valid_scd)):
    
    valid_d_1.append(jobs[id1].result().get_memory(id2)[:, qubit] * 1e-14)
                
    id2 += 1
    if id2 == max_sched:
        id2 = 0
        id1 += 1

np.save(date_dir + '/valid_d_1.npy', valid_d_1)

# Run the validation level 2 program
ind = 0 
jobs = []
while True:
    mn = ind
    mx = min(ind + max_sched, len(valid_scd))
    
    print(f'Now scheduling {mn} to {mx}.')
    
    schedul_program = assemble(valid_scd[mn:mx],
                               backend=backend,
                               meas_level=2,
                               meas_return='single',
                               shots=num_shots)
    
    num_jobs_in_q = backend.status().pending_jobs
    print(f'Pending Jobs: {num_jobs_in_q}')
    
    job = backend.run(schedul_program)
    job_monitor(job)
    jobs.append(job)
    
    ind += max_sched
    
    if ind >= len(valid_scd):
        break
        
# Get the validation level 2 output
id1 = 0
id2 = 0
for itr in range(len(valid_scd)):
    
    valid_d_2.append(jobs[id1].result().get_counts(id2)['0'])
                
    id2 += 1
    if id2 == max_sched:
        id2 = 0
        id1 += 1

np.save(date_dir + '/valid_d_2.npy', valid_d_2)

Now scheduling 0 to 75.
Pending Jobs: 0
Job Status: job incurred error     
Now scheduling 75 to 100.
Pending Jobs: 0
Job Status: job incurred error     


IBMQJobFailureError: 'Unable to retrieve result for job 5ec958cad2d11d001a1a8b90. Job has failed. Use job.error_message() to get more details.'

In [ ]:
data = []
for i in range(len(valid_o_p)):
    data.append(abs(valid_o_p[i] - valid_d_2[i]/num_shots))
    
print(np.median(data))